In [1]:
print("start building")

start building


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)


wikipedia


In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap = 200).split_documents(docs)

In [18]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [19]:
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000204882454D0>, search_kwargs={})

In [20]:
type(retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [21]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriever,"langsmith_search","Search for information about langsmith")

In [22]:
retriver_tool.name

'langsmith_search'

In [23]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [24]:
api_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper)
print(arxiv.name)


arxiv


In [25]:
tools=[wiki,arxiv,retriver_tool]

In [26]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ayush\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000204ECB45300>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object 

In [27]:
type(tools)

list

In [28]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [29]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020489E89110>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020489F31990>, root_client=<openai.OpenAI object at 0x00000204FFED0DD0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020489E8B790>, model_name='gpt-3.5-turbo-0125', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [30]:
type(llm)

langchain_openai.chat_models.base.ChatOpenAI

## Prompt Template

In [36]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\ayush\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [38]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [39]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.m

In [40]:
type(agent)

langchain_core.runnables.base.RunnableSequence

In [41]:
## to run the agent we need agent executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(verbose= True, tools=tools,agent=agent)

In [42]:
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChu

In [43]:
type(agent_executer)

langchain.agents.agent.AgentExecutor

In [44]:
agent_executer.invoke({"input":"Tell me about machine learning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks withMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data. ML algorithms enable computers to learn and improve from experience without being explicitly programmed. This allows them to perform tasks such as data analysis, pattern recognition, and decision-making.

> Finished chain.


{'input': 'Tell me about machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data. ML algorithms enable computers to learn and improve from experience without being explicitly programmed. This allows them to perform tasks such as data analysis, pattern recognition, and decision-making.'}

In [45]:
agent_executer.invoke({"input":"Tell me about how to get the job in data science field?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Data Science'}`


Page: Data science
Summary: Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processing, scientific visualization, algorithms and systems to extract or extrapolate knowledge and insigTo get a job in the field of data science, you typically need a combination of education, skills, and experience. Here are some steps you can take to pursue a career in data science:

1. Education: Most data science roles require a strong educational background in fields such as computer science, statistics, mathematics, or a related field. Consider pursuing a bachelor's or master's degree in data science, computer science, statistics, or a related field.

2. Develop Technical Skills: Data science roles often require proficiency in programming languages like Python, R, SQL, and tools like TensorFlow, Tableau, and Hadoop. Make sure to develop your skills in data analysis, machine lear

{'input': 'Tell me about how to get the job in data science field?',
 'output': "To get a job in the field of data science, you typically need a combination of education, skills, and experience. Here are some steps you can take to pursue a career in data science:\n\n1. Education: Most data science roles require a strong educational background in fields such as computer science, statistics, mathematics, or a related field. Consider pursuing a bachelor's or master's degree in data science, computer science, statistics, or a related field.\n\n2. Develop Technical Skills: Data science roles often require proficiency in programming languages like Python, R, SQL, and tools like TensorFlow, Tableau, and Hadoop. Make sure to develop your skills in data analysis, machine learning, data visualization, and statistical modeling.\n\n3. Gain Practical Experience: Gain practical experience by working on data science projects, internships, or research opportunities. Building a portfolio of projects ca

In [50]:
agent_executer.invoke({"input":"what's the paper 1605.08786 about?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `arxiv` with `{'query': '1605.08786'}`


Published: 2016-05-27
Title: Ideal MHD(-Einstein) Solutions Obeying The Force-Free Condition
Authors: Yi-Zen Chu, Vitaly Vanchurin
Summary: We find two families of analytic solutions to the ideal magnetohydrodynamics
(iMHD) equations, in a class of 4The paper with the identifier 1605.08786 is titled "Ideal MHD(-Einstein) Solutions Obeying The Force-Free Condition" by Yi-Zen Chu and Vitaly Vanchurin. The paper presents two families of analytic solutions to the ideal magnetohydrodynamics (iMHD) equations in a specific class.

> Finished chain.


{'input': "what's the paper 1605.08786 about?",
 'output': 'The paper with the identifier 1605.08786 is titled "Ideal MHD(-Einstein) Solutions Obeying The Force-Free Condition" by Yi-Zen Chu and Vitaly Vanchurin. The paper presents two families of analytic solutions to the ideal magnetohydrodynamics (iMHD) equations in a specific class.'}